## Developing an Information Retrieval System with Spelling Correction and Wildcard Queries

This project aims to enhance the Information Retrieval (IR) system developed in the first assignment by handling
Spelling Correction and Wildcard Queries. This assignment can be completed independently of Project 1. You can
find the data here.


### 1. Document Preprocessing

Your project will begin by reading and preprocessing a collection of text documents. You only need to refer to the dataset as a word list.


In [54]:
import os

import re  # Only for preprocessing

import numpy as np

import marisa_trie

In [4]:
docs_filenames = os.listdir(path="docs")

In [5]:
def clean_text(text):
    """Lowercasing and removing extra characters."""

    text = text.lower()  # Lowercasing

    text = re.sub("[^a-z0-9\s\-]", "", text)  # Removing punctuations

    text = re.sub("\-", " ", text)  # Replacing dash with space

    return text

In [6]:
words_set = set()

for filename in docs_filenames:
    with open(f"docs/{filename}") as f:
        text = f.readline()

        text = clean_text(text)

        words_set.update(text.split())

print(len(words_set))

words_set

1348


{'private',
 'scoop',
 'check',
 'our',
 'gallon',
 'ended',
 'reason',
 'cheaper',
 'ill',
 'middle',
 'idea',
 'busjerry',
 '30',
 'having',
 'quell',
 'joy',
 'gangster',
 'libraries',
 'seashell',
 'reduced',
 'autograph',
 'ending',
 'just',
 'persons',
 'made',
 'clones',
 'lane',
 'hauled',
 'wash',
 'find',
 'worked',
 'exact',
 'parking',
 '70',
 'economy',
 'streambed',
 'accidentally',
 'got',
 'homebuyers',
 'allen',
 'delightful',
 'tim',
 'getting',
 'plastic',
 'tuner',
 'legal',
 'months',
 'chasethey',
 'sell',
 'lucky',
 'thing',
 'prison',
 'refused',
 'wearing',
 'downtown',
 'market',
 '10',
 'saved',
 'antennas',
 'often',
 'fines',
 'carriages',
 'put',
 'altadena',
 'numbers',
 'sizes',
 'trade',
 'whose',
 'greater',
 'line',
 'saw',
 'then',
 'until',
 'near',
 'reads',
 'empty',
 'two',
 'complications',
 'other',
 'theyd',
 'variable',
 'half',
 'cost',
 'roof',
 'insurance',
 'burn',
 'ruined',
 'during',
 'attendance',
 'asked',
 'notify',
 'it',
 'first',

### 2. Spelling Correction:

You will implement a function for isolated spelling correction. Your function needs to
correct an input query using Levenshtein distance based on the words in the list. As the word list derived from
the data is not complete, your function does not work flawlessly for all input queries.


In [7]:
def levenshtein_distance(word1, word2):
    m = np.zeros((len(word1) + 1, len(word2) + 1))

    for i in range(len(word1) + 1):
        m[i, 0] = i

    for j in range(len(word2) + 1):
        m[0, j] = j

    for i in range(1, len(word1) + 1):
        for j in range(1, len(word2) + 1):
            m[i, j] = min(
                m[i - 1, j] + 1,
                m[i, j - 1] + 1,
                m[i - 1, j - 1] + (0 if word1[i-1] == word2[j-1] else 1),
            )

    return m[len(word1), len(word2)]

In [8]:
levenshtein_distance("oslo", "snow")

3.0

In [9]:
def find_nearest_word(word, words_set):
    """Find nearest word using Levenshtein distance method"""

    min_distance = float("inf")
    nearest_word = None

    for w in words_set:
        distance = levenshtein_distance(word, w)

        if distance < min_distance:
            min_distance = distance
            nearest_word = w

        if distance == 0:
            break

    return nearest_word

In [10]:
def spell_checking(query, words_set):
    """Correct the query using words_set and Levenshtein distance method"""

    query = clean_text(query)

    query_words = query.split()

    corrected_words = []

    for word in query_words:
        corrected_word = find_nearest_word(word, words_set)

        corrected_words.append(corrected_word)

    corrected_query = " ".join(corrected_words)

    return corrected_query

In [14]:
spell_checking("festivsl funders", words_set)

'festival founders'

In [15]:
spell_checking("Hello World. You're wild!", words_set)

'sell world your will'

In [16]:
spell_checking("aaa bbb cccc dd", words_set)

'saw bob check did'

In [17]:
spell_checking("probably this wont change", words_set)

'probably this wont change'

### 3. Wildcard Queries

You will implement a function that handles wildcard queries using Permuterm or K-gram
(K=2) indexing. Your function needs to be able to support queries with one or two * symbols. To this end, you
might need to post-filter false positive outcomes. You cannot use Regex.


In [46]:
def inject_dollar_sign(word):
    """Take a world and insert $ in all possible indices"""

    result = []

    for i in range(len(word) + 1):
        new_word = word[:i] + "$" + word[i:]
        result.append(new_word)

    return(result)

In [48]:
inject_dollar_sign("hello")

['$hello', 'h$ello', 'he$llo', 'hel$lo', 'hell$o', 'hello$']

In [55]:
class TrieNode:
    """A node in the trie structure"""

    def __init__(self, char):
        # the character stored in this node
        self.char = char

        # whether this can be the end of a word
        self.is_end = False

        # a dictionary of child nodes
        # keys are characters, values are nodes
        self.children = {}

In [49]:
def build_permuterm_index_tree(words_set):
    trie = marisa_trie.Trie()

    for word in words_set:
        word_list = inject_dollar_sign(word)
        result.extend(word_list)

    return result

In [51]:
build_permuterm_index_tree(words_set)

['$private',
 'p$rivate',
 'pr$ivate',
 'pri$vate',
 'priv$ate',
 'priva$te',
 'privat$e',
 'private$',
 '$scoop',
 's$coop',
 'sc$oop',
 'sco$op',
 'scoo$p',
 'scoop$',
 '$check',
 'c$heck',
 'ch$eck',
 'che$ck',
 'chec$k',
 'check$',
 '$our',
 'o$ur',
 'ou$r',
 'our$',
 '$gallon',
 'g$allon',
 'ga$llon',
 'gal$lon',
 'gall$on',
 'gallo$n',
 'gallon$',
 '$ended',
 'e$nded',
 'en$ded',
 'end$ed',
 'ende$d',
 'ended$',
 '$reason',
 'r$eason',
 're$ason',
 'rea$son',
 'reas$on',
 'reaso$n',
 'reason$',
 '$cheaper',
 'c$heaper',
 'ch$eaper',
 'che$aper',
 'chea$per',
 'cheap$er',
 'cheape$r',
 'cheaper$',
 '$ill',
 'i$ll',
 'il$l',
 'ill$',
 '$middle',
 'm$iddle',
 'mi$ddle',
 'mid$dle',
 'midd$le',
 'middl$e',
 'middle$',
 '$idea',
 'i$dea',
 'id$ea',
 'ide$a',
 'idea$',
 '$busjerry',
 'b$usjerry',
 'bu$sjerry',
 'bus$jerry',
 'busj$erry',
 'busje$rry',
 'busjer$ry',
 'busjerr$y',
 'busjerry$',
 '$30',
 '3$0',
 '30$',
 '$having',
 'h$aving',
 'ha$ving',
 'hav$ing',
 'havi$ng',
 'havin$g'